In [1]:

json = """
[    
    {
        "type":"readers.las",
        "filename":"Test.laz",
        "compression":"laszip"
    },
    {
        "type":"filters.csf"
    },
    {
        "type":"filters.hag_nn"
    },
    {
        "type":"filters.delaunay"
    },
    {
        "type":"filters.faceraster",
        "resolution":5
    },
    {
        "type":"writers.raster",
        "filename":"test.tif"
    }
]
"""

import pdal

pipeline = pdal.Pipeline(json)





In [2]:
pipeline.execute()

13488636